# Variational Encoder Theory Basics

Unsupervised learning is a very important aspect of future artificial intelligences. Researchers have been spending a lot of effort to enable automatic features engineering. Models that recreat sample distributions are called Generative Models. Models such as Restricted Boltzman Machine(RBM) and Variational Encoder are two very famous models in the research field. 

Let $x$ be the observable properties and $z$ be the latent properties. In the model it is assumed that there is a ground truth $p(x|z)$ and the aim is to create a recognition model $q(z|x)$ to approximate $p(z|x)$.

The main technical difficulty for generative model is the intracability of the marginal probability of the observable properties. Being able to track this marginal probability would enable a range of further analysis where a prior $x$ is required. Using technology such as VAE, new models such as Generative Adversal Network(GAN) are made possible.

While RBM uses Gipp Sampling, a Monte Carlo Markov Chain sampling method, to estimate the marginal probability of $x$, VAE uses Monte Carlo variation inference(VI) approach to construct the marginal probability. 

$$log(p_\theta (x))=-D_{KL}(q_\phi (z|x)||p_\theta (z|x))+L (\theta,\phi ; x)$$

As $D_{KL} \geq 0$ so,

$$log(p_\theta (x)) \geq L (\theta,\phi ; x)= E_{q_\phi (z|x)}[log(p_\theta(x,z))-log(q_\phi (z|x))]$$

and so L is also called the evidence lower bound (ELBO).

$log(p_\theta (x))$ is the log likelihood of "evidence" $x$.

$D_{KL}(q_\phi (z|x)||p_\theta (z|x))$ the KL-divergence is actually describing the discrepency between the encoder and decoder. Higher the discrepency, lower is the liklihood of the evidence. By maximising the ELBO, the log-likelihood of the model$log(p_\theta (x))$ is maximised and $D_{KL}$ is minimised.

The ELBO,

i.Monte Carlo form: $$L (\theta,\phi ; x)= E_{q_\phi (z|x)}[log(p_\theta(x,z))-log(q_\phi (z|x))]$$

,can also be generally be written in 2 other forms.

ii. Entropy form:  $$E_{q_\phi (z|x)}[log(p_\theta(x,z))]+H_{q_\phi}$$;

iii. KL divergence form:$$E_{q_\phi (z|x)}[log(p_\theta(x|z)]-D_{KL}({q_\phi(z|x)||p(z|x)})$$.

As $p_\theta(x,z)$ is not trackable, we need to use an estimator to estimate ELBO for optimisation. In order to do that, z is sampled from $q_\phi(z|x)$ to create the ELBO estimator:

$$\frac{1}{L}\Sigma_{l}^{L}  (log(p_\theta(x,z))-log(q_\phi (z|x)))$$

For the entropy form we can have another estimator where the entropy term can be calculated in close form(fully integrated form) instead of Monte Carlo as $q$ is trackable. However, in the case when $q$ is parameterised well to accurately describe marginal $p$ (i.e. when $q$ is flexible enough to accuratly model $p$), the Monte Carlo estimator can approach 0 variance while the closed form entropy version tend to have higher variance. On the other hand, when $q$ cannot describe $p$ well, Monte Carlo estimator tends to give high variance. 

Consider

$$log(p_\theta(x,z))-log(q_\phi (z|x))$$
$$=log(p_\theta(z|x))-log(q_\phi (z|x))+log(p_\theta(x))$$,

when $log(p_\theta(z|x))-log(q_\phi (z|x))$ is small enough, EBLO becomes $log(p_\theta(x))$ and at the same time $D_{KL}$ tends to zero.





#### Reference readings:

Original Paper:https://arxiv.org/pdf/1312.6114.pdf
The original paper describing the VAE design.


Blog: http://andymiller.github.io/2016/12/19/elbo-gradient-estimators.html 
A blog written on VAE. Well analysed different estimators for ELBO optimisation. Also analysed how the structure of its gradient estimators affects training. The analysis on gradient estimator variance is useful.


Other Bayesian inference: http://approximateinference.org/
Further topics related to Bayesian inference and related technology.

# Variational Encoder Application Highlights

Althought the model is well supported by theory, there are lots of details need to be aware of during application. In the theory above, different estimators for ELMO were mentioned. In the coding example below, full Monte Carlo estimation is applied using the KL Divergence format. In the blog, it also suggested the entropy close form or the reduced score format.

### 1. Contradicting Objectives
The cross entropy loss is somehow contradictive with the KL-divergence. As the KL-divergence trying to pack the distribution into the assumed distribution(Gaussian distribution), there is less space the the model to learn to encode the features. The model may get stuck in a local optimum where the KL-divergence is much more optimised than the cross entropy. As a result, manual adjustment may be required to re-adjust the importance between optimising the KL-divergence and the cross entropy loss to optimise the result. After the weighting the two objectives, the KL-loss and cross entropy loss of the converged model have greatly increased.

### 2. Noisy initial latent space
The initial latent space could be a messing as $q$ has not been properly parameterised. Information from the input may be clouded by random sampling. For example, two different sample points(say digit 1 and 5) can get much closer together right after the initialisation.

The decoder may, therefore, fail to retrieve any information from the latent space at all. In this case, the decoder may learn to fit only the KL-divergence objective and then simply make up outputs that equals to the average of all $x$. In this case of mnist, it is assumed that the decoder may be stuck at a local optimum and giving constany output with all the numbers overlapped. 

### 3. Separable training
Training with sampling could be slow because the network would need to learn how to encode the image and deal with the noise from sampling at the same time. To encode deeper features, it would possibly faster for the model to learn the feature encoding first and then train a shallow VAE to transform the encoding space into a variational one. 

### 4. Using log variance instead of variance
It can be shown analytically that it is easier to achieve a stable a log variance output than directly sending variance as output. i.e. variance of the output is smaller.

# Sample Code

In this piece of code we try to implement a simple VAE model with dense layers of neural network. To speed up the training process and make sure the information can be passed along the network, batch normalisation layers are applied between layers. It has been widely confirmed that the applying batch normalisation process between layers can successfully prevent neurons from "dying" as well as speeding up the the training process. 

In this experiment, application of batch normalisation successfully speed up the training process 2+ times. 

In order to visualise and monitor the training, tensorboard code is embedded and in the end, the encoded layer is visualised in embedding projection in tensorboard.

In [6]:
import pandas as pd

In [40]:
import tensorflow as tf
import numpy as np
import os
from tensorflow.examples.tutorials.mnist import input_data    
FLAGS = None
mnist = input_data.read_data_sets('./', one_hot=False)

Extracting ./train-images-idx3-ubyte.gz
Extracting ./train-labels-idx1-ubyte.gz
Extracting ./t10k-images-idx3-ubyte.gz
Extracting ./t10k-labels-idx1-ubyte.gz


In [8]:
import matplotlib.pyplot as plt
%matplotlib inline

# VAE

In [178]:
#Clearing all
tf.reset_default_graph()
#Opening new directory for tensorboard
from datetime import datetime
now = datetime.now()
logdir = 'C:/Users/CK/Documents/tf_logs' + now.strftime("%Y%m%d-%H%M%S") + "/"

#To monitor training through tensorboard
print('TensorBoard CMD:tensorboard --logdir='+logdir)

TensorBoard CMD:tensorboard --logdir=C:/Users/CK/Documents/tf_logs20181011-131207/


In [179]:
learning_rate = 0.01
epochs = 100
batch_size = 800
inputs_ = tf.placeholder(tf.float32, (batch_size, 28, 28, 1), name='inputs')

## Example 1

### Standard VAE

In [180]:
# A sampler to sample variations from the latent space
def sampler(mean,logVar):
    epsilon_=tf.random_normal(shape=[i.value for i in mean.get_shape()])
    return mean + tf.exp(0.5 * logVar) * epsilon_

In [181]:
# A multi-layer coder 
def coder(inputs,layer_units,varScope,batch_normalised=True,activation=tf.nn.relu):
    with tf.variable_scope(varScope):
        layers=[inputs]
        for i in range(0,len(layer_units)):
            print(layers[-1])
            layers=layers+[tf.layers.batch_normalization(layers[-1],training=batch_normalised,axis=-1)]
            layers=layers+[tf.layers.dense(layers[-1],units=layer_units[i],activation=tf.nn.relu)]
    return tf.layers.dense(layers[-1],units=layer_units[-1],activation=activation)

In [1]:
# A variational ecoder using coder input
def vae(inputs,coder,encoder_units,decoder_units,varScope):
    original_shape=inputs.get_shape()
    inputs_flattened=tf.reshape(inputs,shape=(original_shape[0].value,-1))
    with tf.variable_scope(varScope):
        encoded_mean=coder(inputs_flattened,encoder_units,'encoded_mean')
        encoded_logVar=coder(inputs_flattened,encoder_units,'encoded_logVar')
        encoded=sampler(encoded_mean,encoded_logVar)
        logits=coder(encoded,decoder_units,'decoded',activation=None)
        decoded=tf.nn.sigmoid(logits)
        loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=inputs_flattened, logits=logits))
        kl_loss = -0.5*tf.reduce_mean(1 + encoded_logVar - tf.square(encoded_mean) - tf.exp(encoded_logVar))
        with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)):
            adam = tf.train.AdamOptimizer(learning_rate)
            tf.summary.scalar('KL_loss',kl_loss)
            tf.summary.scalar('loss',loss)
            tf.summary.image('Input_Image',inputs)
            tf.summary.image('decoded_Image',tf.reshape(decoded,shape=[i.value for i in original_shape]))

            # Return the encoded code for embedding visualisation, the optimiser for executing training session.
            return encoded,adam.minimize(loss+0.1*kl_loss,var_list= tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES,scope=varScope))   

    
    

In [183]:
#Training model
encoded,opt=vae(inputs_,coder,[196,64,32,16,8],[32,64,196,784],'vae')

saver = tf.train.Saver()
sess = tf.Session()
merged=tf.summary.merge_all()
modelname="VAE"
sess.run(tf.global_variables_initializer())
'''
if(os.path.isfile(modelname+".ckpt.index") ):
    saver.restore(sess, "./"+modelname+".ckpt")
else:
    print('Error when loading model, applying initialisation instead.')
    sess.run(tf.variables_initializer(tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES,scope='vae')))
    print('Model initialised.')
'''

merged=tf.summary.merge_all()
writer = tf.summary.FileWriter(logdir, sess.graph)
for e in range(epochs):
    for ii in range(mnist.train.num_examples//batch_size):
        batch_xs,batch_ys = mnist.train.next_batch(batch_size)
        imgs = batch_xs.reshape((-1, 28, 28, 1)) 
        summary, _, code = sess.run([merged,opt,encoded], feed_dict={inputs_: imgs})
        
        print("Running epochs {}, batch {}".format(e+1,ii+1))
        writer.add_summary(summary, ii+batch_size*e)

    # Add ops to save and restore all the variables.
    saver.save(sess, "./"+modelname+".ckpt")

Tensor("Reshape:0", shape=(800, 784), dtype=float32)
Tensor("vae/encoded_mean/dense/Relu:0", shape=(800, 196), dtype=float32)
Tensor("vae/encoded_mean/dense_1/Relu:0", shape=(800, 64), dtype=float32)
Tensor("vae/encoded_mean/dense_2/Relu:0", shape=(800, 32), dtype=float32)
Tensor("vae/encoded_mean/dense_3/Relu:0", shape=(800, 16), dtype=float32)
Tensor("Reshape:0", shape=(800, 784), dtype=float32)
Tensor("vae/encoded_sd/dense/Relu:0", shape=(800, 196), dtype=float32)
Tensor("vae/encoded_sd/dense_1/Relu:0", shape=(800, 64), dtype=float32)
Tensor("vae/encoded_sd/dense_2/Relu:0", shape=(800, 32), dtype=float32)
Tensor("vae/encoded_sd/dense_3/Relu:0", shape=(800, 16), dtype=float32)
Tensor("vae/add:0", shape=(800, 8), dtype=float32)
Tensor("vae/decoded/dense/Relu:0", shape=(800, 32), dtype=float32)
Tensor("vae/decoded/dense_1/Relu:0", shape=(800, 64), dtype=float32)
Tensor("vae/decoded/dense_2/Relu:0", shape=(800, 196), dtype=float32)
Running epochs 1, batch 1
Running epochs 1, batch 2
Run

Running epochs 4, batch 67
Running epochs 4, batch 68
Running epochs 5, batch 1
Running epochs 5, batch 2
Running epochs 5, batch 3
Running epochs 5, batch 4
Running epochs 5, batch 5
Running epochs 5, batch 6
Running epochs 5, batch 7
Running epochs 5, batch 8
Running epochs 5, batch 9
Running epochs 5, batch 10
Running epochs 5, batch 11
Running epochs 5, batch 12
Running epochs 5, batch 13
Running epochs 5, batch 14
Running epochs 5, batch 15
Running epochs 5, batch 16
Running epochs 5, batch 17
Running epochs 5, batch 18
Running epochs 5, batch 19
Running epochs 5, batch 20
Running epochs 5, batch 21
Running epochs 5, batch 22
Running epochs 5, batch 23
Running epochs 5, batch 24
Running epochs 5, batch 25
Running epochs 5, batch 26
Running epochs 5, batch 27
Running epochs 5, batch 28
Running epochs 5, batch 29
Running epochs 5, batch 30
Running epochs 5, batch 31
Running epochs 5, batch 32
Running epochs 5, batch 33
Running epochs 5, batch 34
Running epochs 5, batch 35
Running ep

Running epochs 9, batch 34
Running epochs 9, batch 35
Running epochs 9, batch 36
Running epochs 9, batch 37
Running epochs 9, batch 38
Running epochs 9, batch 39
Running epochs 9, batch 40
Running epochs 9, batch 41
Running epochs 9, batch 42
Running epochs 9, batch 43
Running epochs 9, batch 44
Running epochs 9, batch 45
Running epochs 9, batch 46
Running epochs 9, batch 47
Running epochs 9, batch 48
Running epochs 9, batch 49
Running epochs 9, batch 50
Running epochs 9, batch 51
Running epochs 9, batch 52
Running epochs 9, batch 53
Running epochs 9, batch 54
Running epochs 9, batch 55
Running epochs 9, batch 56
Running epochs 9, batch 57
Running epochs 9, batch 58
Running epochs 9, batch 59
Running epochs 9, batch 60
Running epochs 9, batch 61
Running epochs 9, batch 62
Running epochs 9, batch 63
Running epochs 9, batch 64
Running epochs 9, batch 65
Running epochs 9, batch 66
Running epochs 9, batch 67
Running epochs 9, batch 68
Running epochs 10, batch 1
Running epochs 10, batch 2
R

Running epochs 13, batch 59
Running epochs 13, batch 60
Running epochs 13, batch 61
Running epochs 13, batch 62
Running epochs 13, batch 63
Running epochs 13, batch 64
Running epochs 13, batch 65
Running epochs 13, batch 66
Running epochs 13, batch 67
Running epochs 13, batch 68
Running epochs 14, batch 1
Running epochs 14, batch 2
Running epochs 14, batch 3
Running epochs 14, batch 4
Running epochs 14, batch 5
Running epochs 14, batch 6
Running epochs 14, batch 7
Running epochs 14, batch 8
Running epochs 14, batch 9
Running epochs 14, batch 10
Running epochs 14, batch 11
Running epochs 14, batch 12
Running epochs 14, batch 13
Running epochs 14, batch 14
Running epochs 14, batch 15
Running epochs 14, batch 16
Running epochs 14, batch 17
Running epochs 14, batch 18
Running epochs 14, batch 19
Running epochs 14, batch 20
Running epochs 14, batch 21
Running epochs 14, batch 22
Running epochs 14, batch 23
Running epochs 14, batch 24
Running epochs 14, batch 25
Running epochs 14, batch 26
R

Running epochs 18, batch 16
Running epochs 18, batch 17
Running epochs 18, batch 18
Running epochs 18, batch 19
Running epochs 18, batch 20
Running epochs 18, batch 21
Running epochs 18, batch 22
Running epochs 18, batch 23
Running epochs 18, batch 24
Running epochs 18, batch 25
Running epochs 18, batch 26
Running epochs 18, batch 27
Running epochs 18, batch 28
Running epochs 18, batch 29
Running epochs 18, batch 30
Running epochs 18, batch 31
Running epochs 18, batch 32
Running epochs 18, batch 33
Running epochs 18, batch 34
Running epochs 18, batch 35
Running epochs 18, batch 36
Running epochs 18, batch 37
Running epochs 18, batch 38
Running epochs 18, batch 39
Running epochs 18, batch 40
Running epochs 18, batch 41
Running epochs 18, batch 42
Running epochs 18, batch 43
Running epochs 18, batch 44
Running epochs 18, batch 45
Running epochs 18, batch 46
Running epochs 18, batch 47
Running epochs 18, batch 48
Running epochs 18, batch 49
Running epochs 18, batch 50
Running epochs 18, b

...

Running epochs 83, batch 31
Running epochs 83, batch 32
Running epochs 83, batch 33
Running epochs 83, batch 34
Running epochs 83, batch 35
Running epochs 83, batch 36
Running epochs 83, batch 37
Running epochs 83, batch 38
Running epochs 83, batch 39
Running epochs 83, batch 40
Running epochs 83, batch 41
Running epochs 83, batch 42
Running epochs 83, batch 43
Running epochs 83, batch 44
Running epochs 83, batch 45
Running epochs 83, batch 46
Running epochs 83, batch 47
Running epochs 83, batch 48
Running epochs 83, batch 49
Running epochs 83, batch 50
Running epochs 83, batch 51
Running epochs 83, batch 52
Running epochs 83, batch 53
Running epochs 83, batch 54
Running epochs 83, batch 55
Running epochs 83, batch 56
Running epochs 83, batch 57
Running epochs 83, batch 58
Running epochs 83, batch 59
Running epochs 83, batch 60
Running epochs 83, batch 61
Running epochs 83, batch 62
Running epochs 83, batch 63
Running epochs 83, batch 64
Running epochs 83, batch 65
Running epochs 83, b

Running epochs 87, batch 53
Running epochs 87, batch 54
Running epochs 87, batch 55
Running epochs 87, batch 56
Running epochs 87, batch 57
Running epochs 87, batch 58
Running epochs 87, batch 59
Running epochs 87, batch 60
Running epochs 87, batch 61
Running epochs 87, batch 62
Running epochs 87, batch 63
Running epochs 87, batch 64
Running epochs 87, batch 65
Running epochs 87, batch 66
Running epochs 87, batch 67
Running epochs 87, batch 68
Running epochs 88, batch 1
Running epochs 88, batch 2
Running epochs 88, batch 3
Running epochs 88, batch 4
Running epochs 88, batch 5
Running epochs 88, batch 6
Running epochs 88, batch 7
Running epochs 88, batch 8
Running epochs 88, batch 9
Running epochs 88, batch 10
Running epochs 88, batch 11
Running epochs 88, batch 12
Running epochs 88, batch 13
Running epochs 88, batch 14
Running epochs 88, batch 15
Running epochs 88, batch 16
Running epochs 88, batch 17
Running epochs 88, batch 18
Running epochs 88, batch 19
Running epochs 88, batch 20
R

Running epochs 92, batch 9
Running epochs 92, batch 10
Running epochs 92, batch 11
Running epochs 92, batch 12
Running epochs 92, batch 13
Running epochs 92, batch 14
Running epochs 92, batch 15
Running epochs 92, batch 16
Running epochs 92, batch 17
Running epochs 92, batch 18
Running epochs 92, batch 19
Running epochs 92, batch 20
Running epochs 92, batch 21
Running epochs 92, batch 22
Running epochs 92, batch 23
Running epochs 92, batch 24
Running epochs 92, batch 25
Running epochs 92, batch 26
Running epochs 92, batch 27
Running epochs 92, batch 28
Running epochs 92, batch 29
Running epochs 92, batch 30
Running epochs 92, batch 31
Running epochs 92, batch 32
Running epochs 92, batch 33
Running epochs 92, batch 34
Running epochs 92, batch 35
Running epochs 92, batch 36
Running epochs 92, batch 37
Running epochs 92, batch 38
Running epochs 92, batch 39
Running epochs 92, batch 40
Running epochs 92, batch 41
Running epochs 92, batch 42
Running epochs 92, batch 43
Running epochs 92, ba

Running epochs 96, batch 31
Running epochs 96, batch 32
Running epochs 96, batch 33
Running epochs 96, batch 34
Running epochs 96, batch 35
Running epochs 96, batch 36
Running epochs 96, batch 37
Running epochs 96, batch 38
Running epochs 96, batch 39
Running epochs 96, batch 40
Running epochs 96, batch 41
Running epochs 96, batch 42
Running epochs 96, batch 43
Running epochs 96, batch 44
Running epochs 96, batch 45
Running epochs 96, batch 46
Running epochs 96, batch 47
Running epochs 96, batch 48
Running epochs 96, batch 49
Running epochs 96, batch 50
Running epochs 96, batch 51
Running epochs 96, batch 52
Running epochs 96, batch 53
Running epochs 96, batch 54
Running epochs 96, batch 55
Running epochs 96, batch 56
Running epochs 96, batch 57
Running epochs 96, batch 58
Running epochs 96, batch 59
Running epochs 96, batch 60
Running epochs 96, batch 61
Running epochs 96, batch 62
Running epochs 96, batch 63
Running epochs 96, batch 64
Running epochs 96, batch 65
Running epochs 96, b

Running epochs 100, batch 54
Running epochs 100, batch 55
Running epochs 100, batch 56
Running epochs 100, batch 57
Running epochs 100, batch 58
Running epochs 100, batch 59
Running epochs 100, batch 60
Running epochs 100, batch 61
Running epochs 100, batch 62
Running epochs 100, batch 63
Running epochs 100, batch 64
Running epochs 100, batch 65
Running epochs 100, batch 66
Running epochs 100, batch 67
Running epochs 100, batch 68


## Graph


In [ ]:
#To show the tensorBoard
from IPython.display import clear_output, Image, display, HTML

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))
show_graph(tf.get_default_graph().as_graph_def())

# Embeddings(code) visualisation

In [184]:
def create_sprite_image(images):
    """Returns a sprite image consisting of images passed as argument. Images should be count x width x height"""
    if isinstance(images, list):
        images = np.array(images)
    img_h = images.shape[1]
    img_w = images.shape[2]
    n_plots = int(np.ceil(np.sqrt(images.shape[0])))
    
    
    spriteimage = np.ones((img_h * n_plots ,img_w * n_plots ))
    
    for i in range(n_plots):
        for j in range(n_plots):
            this_filter = i * n_plots + j
            if this_filter < images.shape[0]:
                this_img = images[this_filter]
                spriteimage[i * img_h:(i + 1) * img_h,
                  j * img_w:(j + 1) * img_w] = this_img
    
    return spriteimage

def vector_to_matrix_mnist(mnist_digits):
    """Reshapes normal mnist digit (batch,28*28) to matrix (batch,28,28)"""
    return np.reshape(mnist_digits,(-1,28,28))

def invert_grayscale(mnist_digits):
    """ Makes black white, and white black """
    return 1-mnist_digits

In [185]:
to_visualise = batch_xs
to_visualise = vector_to_matrix_mnist(to_visualise)
to_visualise = invert_grayscale(to_visualise)

sprite_image = create_sprite_image(to_visualise)
plt.imsave(os.path.join(logdir,'mnistdigits.png'),sprite_image,cmap='gray')

In [186]:
with open(os.path.join(logdir,'metadata.tsv'),'w') as f:
    f.write("Index\tLabel\n")
    for index,label in enumerate(batch_ys):
        f.write("%d\t%d\n" % (index,label))

In [187]:
# Embeddings logging for Tensorboard
from tensorflow.contrib.tensorboard.plugins import projector
config = projector.ProjectorConfig()
embedding_var=tf.Variable(code,dtype=tf.float32,name='embeddings')
embedding = config.embeddings.add()
embedding.tensor_name = embedding_var.name
sess.run(embedding_var.initializer)
summary_writer = tf.summary.FileWriter(logdir)
# Specify where you find the metadata
embedding.metadata_path = os.path.join(logdir,'metadata.tsv') #'metadata.tsv'

# Specify where you find the sprite (we will create this later)
embedding.sprite.image_path = os.path.join(logdir,'mnistdigits.png') #'mnistdigits.png'
embedding.sprite.single_image_dim.extend([28,28])

# Say that you want to visualise the embeddings
projector.visualize_embeddings(summary_writer, config)
saver = tf.train.Saver([embedding_var])
saver.save(sess, os.path.join(logdir, 'embeddings.ckpt'))

'C:/Users/CK/Documents/tf_logs20181011-131207/embeddings.ckpt'